In [115]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [116]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [117]:
dig=load_digits()

In [118]:
onehot_target=pd.get_dummies(dig.target)

In [119]:
x_train,x_val,y_train,y_val=train_test_split(dig.data,onehot_target,test_size=0.1)

In [120]:
def sigmoid(s):
    return 1/(1+np.exp(-s)) 
def sigmoid_derv(s):
    return s*(1-s)


In [121]:
def softmax(s):
    exps=np.exp(s-np.max(s,axis=1,keepdims=True))
    return exps/np.sum(exps,axis=1,keepdims=True )

In [122]:
def cross_entropy(pred,real):
    n_samples=real.shape[0]
    res=pred-real
    return res/n_samples

In [123]:
def error(pred,real):
    n_samples=real.shape[0]
    logp=-np.log(pred[np.arange(n_samples),real.argmax(axis=1)])
    loss=np.sum(logp)/n_samples
    return loss

#error

In [124]:

class MyNN:
    def __init__(self,x,y):
        #initialising weights and biases
        self.x=x

        neurons=128 #aka a
        self.lr=0.5
        ip_dim=x.shape[1]
        op_dim=y.shape[1]
        self.w1=np.random.randn(ip_dim,neurons)
        self.b1=np.zeros((1,neurons))
        self.w2=np.random.randn(neurons,neurons)
        self.b2=np.zeros((1,neurons))
        self.w3=np.random.randn(neurons,op_dim)
        self.b3=np.zeros((1,op_dim))
        self.y=y
        
    def feedforward(self):
        z1=np.dot(self.x,self.w1)+self.b1
        self.a1=sigmoid(z1)
        z2=np.dot(self.a1,self.w2)+self.b2
        self.a2=sigmoid(z2)
        z3=np.dot(self.a2,self.w3)+self.b3
        self.a3=softmax(z3)#softmax returns values betn 0 to1 which have sum 1,sum 1 is necessary as given number is A NUBER BETWEEEN 0 TO 9

    def backprop(self):
        loss=error(self.a3,self.y)
        a3_delta=cross_entropy(self.a3,self.y)#w3
        z2_delta=np.dot(a3_delta,self.w3.T)
        a2_delta=z2_delta*sigmoid_derv(self.a2)#w2
        z1_delta=np.dot(a2_delta,self.w2.T)
        a1_delta=z1_delta*sigmoid_derv(self.a1)


        self.w3 -= self.lr*np.dot(self.a2.T,a3_delta)
        self.b3 -= self.lr*np.sum(a3_delta,axis=0,keepdims=True)
        self.w2 -= self.lr*np.dot(self.a1.T,a2_delta)
        self.b2 -= self.lr*np.sum(a2_delta,axis=0)
        self.w1 -= self.lr*np.dot(self.x.T,a1_delta)
        self.b1 -= self.lr*np.sum(a1_delta,axis=0)
        
    def predict(self,data):
        self.x=data
        self.feedforward()
        return self.a3.argmax()


In [125]:
model = MyNN(x_train,np.array(y_train))

In [126]:
epochs=1500
for x in range(epochs):
    model.feedforward()
    model.backprop()


In [127]:
def get_acc(x,y):
    acc=0
    for xx,yy in zip(x,y):
        s=model.predict(xx)
        if s==np.argmax(yy):
            acc +=1
    return acc/len(x)*100

In [128]:
get_acc(x_val, np.array(y_val))

93.33333333333333